In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: 대학생

In [3]:
#http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%8C%80%ED%95%99%EC%83%9D&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%8C%80%ED%95%99%EC%83%9D

In [4]:
paper_link_list = []
#50
for i in range(50):
    before_url = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%8C%80%ED%95%99%EC%83%9D&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%8C%80%ED%95%99%EC%83%9D"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

대학생의 성희롱 및 성평등 인식 수준 및 영향 요인
대학급식의 친환경 농산물 이용에 대한 전남ㆍ경남지역 대학생의 가격민감성 비교
대학생의 건강관련 생활태도와 모유수유의지에 관한 연구
대학생의 영양표시 확인 여부에 따른 영양표시에 대한 지식, 인식 및 유용성 자각 비교
대학생이 지각한 가족탄력성과 한국판 동적가족화 반응특성 관계 연구
대학생 진로역량 척도 개발
대학생의 외모만족도와 대학생활적응능력이 자아존중감에 미치는 영향
대학생의 고용가능성 진단 도구 개발
대학생의 성적 자기주장에 영향을 미치는 요인
대학생들의 패스트푸드 소비 형태와 행동에 대한 연구
10
대학생의 진로적응력과 자기수용, 강점인식, 강점활용의 구조적 관계
대학생의 진로적응력과 주도적 성향, 진로탐색행동, 진로의사결정 자기효능감의 구조관계
대학생의 강점인식과 진로준비행동의 관계
대학생의 건강지각, 자기효능감, 사회적 지지가 건강증진행위에 미치는 영향
대학생의 환경친화적 소비의식과 자원절약행동에 관한 연구
대학생의 성격강점과 의사소통유형이 대인관계만족도에 미치는 영향
대학생의 수면과 감각처리 특성의 상관관계
간호 대학생이 느끼는 간호사 이미지와 전공만족도가 간호전문직관에 미치는 영향
서울지역 대학생의 자발적 졸업연기에 대한 질적 연구
간호보건계열 대학생의 전공만족도, 진로성숙도가 취업스트레스에 미치는 영향
20
보건계열과 비보건계열 대학생의 자아존중감과 건강증진행위에 관한 비교 연구 : 응급구조과와 공과계열 학생 간 비교
DBT 음악치료가 대학생의 심리적 안녕감과 자아탄력성 향상에 미치는 영향
학술논문 : 화예조형예술 작품에 대한 상징성 인식 분석 -남녀 대학생을 대상으로-
저널쓰기를 병행한 집단예술치료가 대학생의 자아존중감에 미치는 영향
대학생의 색선호도와 낙관성과의 관계
대학생의 자아존중감이 대인관계능력에 미치는 영향 : 정서조절방략의 매개 효과
대학생의 정신건강과 건강위험행위의 관련성
대학생의 진로결정자기효능감과 셀프리더십, 개인이 인식한 문제해결력 및 진로동기의 인과적 관계
섬유염색공예를 

건강증진 프로그램이 체지방율 비만 대학생의 대사증후군 위험요인, 건강 효능감과 내적 건강통제위에 미치는 효과
강원 영서지역 남, 여 대학생의 건강 기능성 식품인 인삼 및 인삼제품에 대한 인식도 조사
원저 : 공중보건학을 수강하는 대학생의 건강증진생활 실천도
연구논문(硏究論文) : 대학생의 중도탈락에 미치는 교우관계의 영향력과 학교적응의 매개효과
충남 일부 지역 대학생의 식사의 질, 식행동 및 에너지 평형상태 조사 연구
연구논문 : 미용전공 대학생의 전공만족도가 진로결정효능감에 미치는 영향
연구논문 : 미용전공 대학생의 봉사활동 만족도와 전공만족도에 관한 연구
연구논문 : 남녀 대학생의 성별에 따른 미용관리 및 체중관리에 대한 영향요인 분석
미국에 거주하는 한인 대학생들의 출생지와 미국 내 거주기간에 따른 체형인식 및 섭식장애에 관한 연구
원저 : 일부 대학생의 구취자각에 따른 건강상태 및 건강행동
230
아동학, 유아교육 전공 대학생의 읽기 실태에 관한 조사연구
연구논문 : 미용전공 대학생의 직업가치관이 진로성숙도에 미치는 영향
심리적 탄력성과 자기 통제력이 대학생의 자살생각에 미치는 영향
단기간 현미채식과 옥외걷기운동이 대학생의 신체조성 및 혈중지질수준에 미치는 영향
국내 한국인 및 중국인 대학생의 라이프스타일별 커피 만족도 선행요인 분석
원저 : 20대 대학생들의 상악중절치 색조와 구강보건 및 식습관 관련 특성
연구논문 : 미용관련학과 대학생들의 자기효능감과 자아존중감 연구
연구논문 : 우리나라 일부 대학생의 건강기능식품 섭취 실태 및 섭취 요인 연구
연구논문 : 남자 대학생의 헤어, 피부, 메이크업에 관한 의식이 외모관리행동에 미치는 영향
연구논문 : 원주지역 대학생들의 음주실태 및 음주문화에 대한 인식
240
원저 : 일부 대학생들의 치주건강에 대한 주관적 인식과 혈구수치와의 상관관계
연구논문 : "그린 하모니"-대학생의 전인건강을 위한 원예치료 프로그램
노출 시나리오를 이용한 대학생들의 유해 공기오염물질 노출 추정
보문 : 남자 대학생의 의복관여, 유

마찰에 의한 통증 입력이 작업자의 상지 고유수용성 감각에 미치는 영향
타인의 인정에 대한 과도한 요구 신념과 정서적 고통 감내력이 사회적 수행불안에 미치는 영향
연구논문 : 5분과 ; 산림 내 호수 주변에서의 산책과 경관감상이 인체에 미치는 생리적, 심리적 영향
정시안과 비정시안의 건성안 설문조사 (McMonnies - DEQ)로 알아본 자각적 불편감과 순목횟수
중국 전통원림의 치석피도(置石被度)에 따른 원로경관의 시지각적 특성 분석 -한국인과 중국인 시지각 비교를 중심으로-
교육매체 활용 : 가상 교육용 전자장 해석 프로그램 개발
언론 수용자의 공정성 평가가 매체 이용에 미친 영향 분석
긍정적인 사회적 상황에 대한 해석편향과 사회불안의 관계 : 타인기준-자기 효능감 불일치의 매개효과
심리극집단프로그램이 성매매경험여성의 외상 후 스트레스와 자아존중감에 미치는 영향
온라인 브랜드커뮤니티가 소비자 태도에 미치는 영향 : 온라인 대학 브랜드 선택을 중심으로
420
긍정적 평가에 대한 두려움과 사회불안의 관계 : 긍정적인 사회적 상황에 대한 해석편향의 매개효과
단안용 웨어러블 디바이스가 단기적으로 조절에 미치는 영향
여가활동으로서의 무도스포츠 역할 재고를 위한 고찰
효율적인 산업체 현장실습 운영에 대한 연구
대학시설들의 지역사회 연계에 따른 이용자 만족도 및 도심활성화에 미치는 영향 분석 : 대학로 오프 캠퍼스 대학시설들을 대상으로
실천공학 교수법 : Kolb학습유형검사의 한글버전 개발
대구시 동대구로의 경관조명에 관한 주민의식 조사 : 동대구로 경관조명의 디자인 특성에 관한 연구
원저 : RGP 렌즈의 형상에 따른 파면수차 분석
경계선 성격장애 성향자의 이분법적 사고와 단일차원적 사고
근시 교정에 따른 수평사위도 변화
430
텔레비전 프로그램의 선정성 심의기준에 대한 수용자 인식 연구
우울취약성과 자살사고의 관계 : 속박패배감의 매개효과
신입사원 채용시 사회봉사실적 반영방안
Contact Lens Dry Eye Questionnaire(CLDEQ)를 이용한 콘택트렌

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=4e090561510b03f7ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=48c16df9dfb868e6ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=33c865a105e6ff0fffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=3ff0aa1c0b012585ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e8ce53f7353706b2ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e88a545be561a8a047de9c1710b0298d',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=969fbf17e518fa587f7a54760bb41745',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d4824167cad58e25ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=fd931c3231c7297fffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

499

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|████████████████████████████████████████████████████████████████████████████████████████| 499/499 [00:00<?, ?it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=4e090561510b03f7ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=48c16df9dfb868e6ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=33c865a105e6ff0fffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=3ff0aa1c0b012585ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e8ce53f7353706b2ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e88a545be561a8a047de9c1710b0298d',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=969fbf17e518fa587f7a54760bb41745',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d4824167cad58e25ffe0bdc3ef48d419',
 'http://www.ris

In [9]:
len(paper_url)

499

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./1_대학생.xlsx')
    
print("Running Time : ", time.time() - start)

0 paper clear
1 paper clear
2 paper clear
3 paper clear
4 paper clear
5 paper clear
6 paper clear
7 paper clear
8 paper clear
9 paper clear
10 paper clear
11 paper clear
12 paper clear
13 paper clear
14 paper clear
15 paper clear
16 paper clear
17 paper clear
18 paper clear
19 paper clear
20 ERROR
21 paper clear
22 paper clear
23 paper clear
24 paper clear
25 paper clear
26 paper clear
27 paper clear
28 paper clear
29 paper clear
30 paper clear
31 paper clear
32 paper clear
33 paper clear
34 paper clear
35 paper clear
36 paper clear
37 paper clear
38 paper clear
39 paper clear
40 paper clear
41 ERROR
42 paper clear
43 paper clear
44 paper clear
45 paper clear
46 paper clear
47 paper clear
48 paper clear
49 paper clear
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 paper clear
56 paper clear
57 paper clear
58 paper clear
59 paper clear
60 paper clear
61 paper clear
62 paper clear
63 paper clear
64 paper clear
65 paper clear
66 ERROR
67 paper clear
68 paper